# Format DataFrame

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
train_df = pd.DataFrame(data.data, columns=data.feature_names)
train_df["diagnosis"] = data.target

print(train_df.shape)
train_df.head()

(569, 31)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


# Set Up Environment

In [2]:
from hyperparameter_hunter import Environment, CVExperiment
from sklearn.model_selection import StratifiedKFold

env = Environment(
    train_dataset=train_df,
    results_path="HyperparameterHunterAssets",
    target_column="diagnosis",
    metrics_map=["roc_auc_score"],
    cross_validation_type=StratifiedKFold,
    cv_params=dict(n_splits=10, shuffle=True, random_state=32),
)

Cross-Experiment Key:   'LOKr-L2lRGe8e8l0E-8SY5FEcl2u7dwBRtrl36HDjvQ='


Now that HyperparameterHunter has an active `Environment`, we can do two things:

# 1. Perform Experiments

In [3]:
from xgboost import XGBClassifier

experiment = CVExperiment(
    model_initializer=XGBClassifier,
    model_init_params=dict(objective="reg:linear", max_depth=3, n_estimators=100, subsample=0.5),
    model_extra_params=dict(
        fit=dict(
            eval_set=[(env.train_input, env.train_target), (env.validation_input, env.validation_target)],
            early_stopping_rounds=5,
            eval_metric="mae",
        ),
    ),
)

<18:54:43> Validated Environment:  'LOKr-L2lRGe8e8l0E-8SY5FEcl2u7dwBRtrl36HDjvQ='
<18:54:43> Initialized Experiment: 'c3ddb52c-ac47-46bd-9b11-f6407f7ca611'
<18:54:43> Hyperparameter Key:     'ntCovu5ufjSeQtJZkSxwzO0oDL0s5aq4BstpfidOTZE='
<18:54:43> 
<18:54:43> F0.0 AVG:   OOF(roc_auc_score=0.95455)  |  Time Elapsed: 0.04106 s
<18:54:43> F0.1 AVG:   OOF(roc_auc_score=0.96338)  |  Time Elapsed: 0.04396 s
<18:54:44> F0.2 AVG:   OOF(roc_auc_score=0.90079)  |  Time Elapsed: 0.04339 s
<18:54:44> F0.3 AVG:   OOF(roc_auc_score=0.97222)  |  Time Elapsed: 0.03908 s
<18:54:44> F0.4 AVG:   OOF(roc_auc_score=0.97619)  |  Time Elapsed: 0.03897 s
<18:54:44> F0.5 AVG:   OOF(roc_auc_score=0.90079)  |  Time Elapsed: 0.03904 s
<18:54:44> F0.6 AVG:   OOF(roc_auc_score=0.98611)  |  Time Elapsed: 0.03699 s
<18:54:44> F0.7 AVG:   OOF(roc_auc_score=0.98571)  |  Time Elapsed: 0.03822 s
<18:54:44> F0.8 AVG:   OOF(roc_auc_score=0.97619)  |  Time Elapsed: 0.03699 s
<18:54:44> F0.9 AVG:   OOF(roc_auc_score=1.00000

# 2. Hyperparameter Optimization

In [4]:
from hyperparameter_hunter import BayesianOptimization, Real, Integer, Categorical

optimizer = BayesianOptimization(iterations=30, random_state=1337)

optimizer.set_experiment_guidelines(
    model_initializer=XGBClassifier,
    model_init_params=dict(
        objective="reg:linear",
        max_depth=Integer(2, 20),
        learning_rate=Real(0.0001, 0.5),
        subsample=0.5,
        booster=Categorical(["gbtree", "dart"]),
    ),
    model_extra_params=dict(
        fit=dict(
            eval_set=[(env.train_input, env.train_target), (env.validation_input, env.validation_target)],
            early_stopping_rounds=5,
            eval_metric=Categorical(["auc", "mae"]),
        ),
    ),
)

optimizer.go()

Validated Environment with key: "LOKr-L2lRGe8e8l0E-8SY5FEcl2u7dwBRtrl36HDjvQ="
Saved Result Files
___________________________________________________________________________________________________________
 Step |       ID |   Time |      Value |   (fit, eval_metric) |   booster |   learning_rate |   max_depth | 
Experiments matching cross-experiment key/algorithm: 1
Experiments fitting in the given space: 1
Experiments matching current guidelines: 1
    0 | c3ddb52c | 00m00s |    0.96145 |                  mae |    gbtree |          0.1000 |           3 | 
Hyperparameter Optimization
___________________________________________________________________________________________________________
 Step |       ID |   Time |      Value |   (fit, eval_metric) |   booster |   learning_rate |   max_depth | 
    1 | 7239620d | 00m00s |    0.95298 |                  mae |      dart |          0.1864 |          17 | 
    2 | a54f1e46 | 00m00s |    0.94258 |                  mae |    gbtree |       

Notice, `optimizer` recognizes our earlier `experiment`'s hyperparameters fit inside the search space/guidelines set for `optimizer`.

Then, when optimization is started, it automatically learns from `experiment`'s results - without any extra work for us!